In [1]:
import os
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.76


In [3]:
# use this code to set up config file
subscription_id =''
resource_group =''
workspace_name = ''

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print('Workspace configuration succeeded. You are all set!')
except:
   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace configuration succeeded. You are all set!


# Compute Environment
Update the cluster to the existing compute name or if there is not an existing compute it will create one for you with the provisioning_configuration specificed.

In [4]:
cluster = 'trainingCompute'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC12', min_nodes=1, max_nodes=5)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

Found existing compute target


# Run experiment 

In [5]:
from six.moves.urllib.request import urlretrieve
from pathlib import Path

data_path = 'data'
mnist_file = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
target = Path(data_path).joinpath('mnist.npz')
if not os.path.exists(target):
    os.mkdir(data_path)
    print('downloading {} ...'.format(mnist_file))
    file, output = urlretrieve(mnist_file, target)

downloading https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz ...


In [6]:
ds = ws.get_default_datastore()
mnist_data = ds.upload(src_dir = '.', target_path = 'mnist', show_progress = True)

Uploading an estimated of 9 files
Uploading ./requirements.txt
Uploading ./Untitled.ipynb
Uploading ./digits.yml
Uploading ./train.py
Uploading ./amlcallback.py
Uploaded ./requirements.txt, 1 files out of an estimated total of 9
Uploading ./score.py
Uploading ./.ipynb_checkpoints/Untitled-checkpoint.ipynb
Uploaded ./score.py, 2 files out of an estimated total of 9
Uploaded ./digits.yml, 3 files out of an estimated total of 9
Uploading ./data/mnist.npz
Uploaded ./Untitled.ipynb, 4 files out of an estimated total of 9
Uploading ./.azureml/config.json
Uploaded ./train.py, 5 files out of an estimated total of 9
Uploaded ./amlcallback.py, 6 files out of an estimated total of 9
Uploaded ./.azureml/config.json, 7 files out of an estimated total of 9
Uploaded ./.ipynb_checkpoints/Untitled-checkpoint.ipynb, 8 files out of an estimated total of 9
Uploaded ./data/mnist.npz, 9 files out of an estimated total of 9
Uploaded 9 files


In [8]:
# run the same was as above
script_params={
    '--data': mnist_data.as_mount()
}

# Create and run experiment
mnist = Experiment(ws, 'mnist_lab')
estimator = Estimator(source_directory='.',
                      compute_target=compute,
                      entry_script='train.py',
                      use_gpu=True,
                      pip_packages=['matplotlib', 'pathlib', 'tensorflow-gpu'])

'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


In [9]:
run = mnist.submit(estimator)
RunDetails(run).show()

'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

## Retrieve and register model

In [12]:
print(run.get_file_names())

['azureml-logs/20_image_build_log.txt']


In [14]:
# download model
run.download_file(name='outputs/latest.hdf5', output_file_path='outputs')
#register model
model = run.register_model(model_name='mnist_lab',
                           model_path='outputs/latest.hdf5')
print(model.name, model.id, model.version, sep='\t')

mnist_lab	mnist_lab:1	1


## Prepare deployment environment

In [15]:
myenv = CondaDependencies()

myenv.add_pip_package('tensorflow-gpu')

with open('digits.yml','w') as f:
    print('Writing out {}'.format('digits.yml'))
    f.write(myenv.serialize_to_string())
    print('Done!')

Writing out digits.yml
Done!


In [16]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               tags={"data": "MNIST",
                                                     "method": "tf20"},
                                               description='Predict MNIST with TF20')

## Deploy 

In [17]:
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="digits.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='tf2-mnist-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Running...................................................
Succeeded
Image creation operation finished for image tf2-mnist-svc:1, operation "Succeeded"
Running.......................
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test deployment

In [18]:
scoring_uri = service.scoring_uri


In [19]:
import json
import requests

img = '0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,67,232,39,0,0,0,0,0,0,0,0,0,62,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120,180,39,0,0,0,0,0,0,0,0,0,126,163,0,0,0,0,0,0,0,0,0,0,0,0,0,2,153,210,40,0,0,0,0,0,0,0,0,0,220,163,0,0,0,0,0,0,0,0,0,0,0,0,0,27,254,162,0,0,0,0,0,0,0,0,0,0,222,163,0,0,0,0,0,0,0,0,0,0,0,0,0,183,254,125,0,0,0,0,0,0,0,0,0,46,245,163,0,0,0,0,0,0,0,0,0,0,0,0,0,198,254,56,0,0,0,0,0,0,0,0,0,120,254,163,0,0,0,0,0,0,0,0,0,0,0,0,23,231,254,29,0,0,0,0,0,0,0,0,0,159,254,120,0,0,0,0,0,0,0,0,0,0,0,0,163,254,216,16,0,0,0,0,0,0,0,0,0,159,254,67,0,0,0,0,0,0,0,0,0,14,86,178,248,254,91,0,0,0,0,0,0,0,0,0,0,159,254,85,0,0,0,47,49,116,144,150,241,243,234,179,241,252,40,0,0,0,0,0,0,0,0,0,0,150,253,237,207,207,207,253,254,250,240,198,143,91,28,5,233,250,0,0,0,0,0,0,0,0,0,0,0,0,119,177,177,177,177,177,98,56,0,0,0,0,0,102,254,220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,254,137,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,254,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,255,94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,254,96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,254,153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169,255,153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,254,153,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0'
data = { 'image': img }

headers = {'Content-Type': 'application/json'}
resp = requests.post(scoring_uri, json.dumps(data), headers=headers)


In [20]:
pred = json.loads(resp.text)
print('{}: {}'.format('Prediction', pred['prediction']))
print('{}: {}'.format('Elapsed', pred['time']))
print('Scores:')
scores = pred['scores'][0]
for i in range(len(scores)):
    print('   {}: {}'.format(i, scores[i]))

Prediction: 4
Elapsed: 0.262721
Scores:
   0: 5.966259867209406e-17
   1: 2.9407381418877776e-08
   2: 3.4450222622522952e-12
   3: 1.5817362813952927e-14
   4: 0.9999998807907104
   5: 1.1679420519367274e-14
   6: 1.8137680485134922e-13
   7: 1.29070174686996e-11
   8: 2.776833762929698e-10
   9: 1.0097365077399445e-07
